In [3]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
#
# Copyright 2020 Confluent Inc.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


# A simple example demonstrating use of JSONSerializer.

import argparse
from uuid import uuid4
from six.moves import input
from confluent_kafka import Producer
from confluent_kafka.serialization import StringSerializer, SerializationContext, MessageField
from confluent_kafka.schema_registry import SchemaRegistryClient
from confluent_kafka.schema_registry.json_schema import JSONSerializer
#from confluent_kafka.schema_registry import *
import pandas as pd
from typing import List

FILE_PATH = "/home/pradeep.k@zucisystems.com/workspace_working_Repo/Message_Queue/Data/car_dataset.csv"
columns=['car_name', 'brand', 'model', 'vehicle_age', 'km_driven', 'seller_type',
       'fuel_type', 'transmission_type', 'mileage', 'engine', 'max_power',
       'seats', 'selling_price']

API_KEY = 'NFABB2LKRMYGAPL7'
ENDPOINT_SCHEMA_URL  = 'https://psrc-mw0d1.us-east-2.aws.confluent.cloud'
API_SECRET_KEY = '240rDHD/DK10Q9EqPZoJjJTsQ0D+k8jyx3so3RyKgcNShXchncJiJq3Nj0cx+ohL'
BOOTSTRAP_SERVER = 'pkc-6ojv2.us-west4.gcp.confluent.cloud:9092'
SECURITY_PROTOCOL = 'SASL_SSL'
SSL_MACHENISM = 'PLAIN'
SCHEMA_REGISTRY_API_KEY = 'DIH3DYE4IT6KIDRM'
SCHEMA_REGISTRY_API_SECRET = '3MWjS0SF145fAMQy+vZ610Om5k5pNWTk798iGwOiZnGgQdgKLPLasKyBd2cABhUd'


def sasl_conf():

    sasl_conf = {'sasl.mechanism': SSL_MACHENISM,
                 # Set to SASL_SSL to enable TLS support.
                #  'security.protocol': 'SASL_PLAINTEXT'}
                'bootstrap.servers':BOOTSTRAP_SERVER,
                'security.protocol': SECURITY_PROTOCOL,
                'sasl.username': API_KEY,
                'sasl.password': API_SECRET_KEY
                }
    return sasl_conf



def schema_config():
    return {'url':ENDPOINT_SCHEMA_URL,
    
    'basic.auth.user.info':f"{SCHEMA_REGISTRY_API_KEY}:{SCHEMA_REGISTRY_API_SECRET}"

    }


class Car:   
    def __init__(self,record:dict):
        for k,v in record.items():
            setattr(self,k,v)
        
        self.record=record
   
    @staticmethod
    def dict_to_car(data:dict,ctx):
        return Car(record=data)

    def __str__(self):
        return f"{self.record}"


def get_car_instance(file_path):
    df=pd.read_csv(file_path)
    df=df.iloc[:,1:]
    cars:List[Car]=[]
    for data in df.values:
        car=Car(dict(zip(columns,data)))
        cars.append(car)
        yield car

def car_to_dict(car:Car, ctx):
    """
    Returns a dict representation of a User instance for serialization.
    Args:
        user (User): User instance.
        ctx (SerializationContext): Metadata pertaining to the serialization
            operation.
    Returns:
        dict: Dict populated with user attributes to be serialized.
    """

    # User._address must not be serialized; omit from dict
    return car.record


def delivery_report(err, msg):
    """
    Reports the success or failure of a message delivery.
    Args:
        err (KafkaError): The error that occurred on None on success.
        msg (Message): The message that was produced or failed.
    """

    if err is not None:
        print("Delivery failed for User record {}: {}".format(msg.key(), err))
        return
    print('User record {} successfully produced to {} [{}] at offset {}'.format(
        msg.key(), msg.topic(), msg.partition(), msg.offset()))


def main(topic):

    schema_str = """
    {
  "$id": "http://example.com/myURI.schema.json",
  "$schema": "http://json-schema.org/draft-07/schema#",
  "additionalProperties": false,
  "description": "Sample schema to help you get started.",
  "properties": {
    "brand": {
      "description": "The type(v) type is used.",
      "type": "string"
    },
    "car_name": {
      "description": "The type(v) type is used.",
      "type": "string"
    },
    "engine": {
      "description": "The type(v) type is used.",
      "type": "number"
    },
    "fuel_type": {
      "description": "The type(v) type is used.",
      "type": "string"
    },
    "km_driven": {
      "description": "The type(v) type is used.",
      "type": "number"
    },
    "max_power": {
      "description": "The type(v) type is used.",
      "type": "number"
    },
    "mileage": {
      "description": "The type(v) type is used.",
      "type": "number"
    },
    "model": {
      "description": "The type(v) type is used.",
      "type": "string"
    },
    "seats": {
      "description": "The type(v) type is used.",
      "type": "number"
    },
    "seller_type": {
      "description": "The type(v) type is used.",
      "type": "string"
    },
    "selling_price": {
      "description": "The type(v) type is used.",
      "type": "number"
    },
    "transmission_type": {
      "description": "The type(v) type is used.",
      "type": "string"
    },
    "vehicle_age": {
      "description": "The type(v) type is used.",
      "type": "number"
    }
  },
  "title": "SampleRecord",
  "type": "object"
}
    """
    schema_registry_conf = schema_config()
    schema_registry_client = SchemaRegistryClient(schema_registry_conf)

    string_serializer = StringSerializer('utf_8')
    json_serializer = JSONSerializer(schema_str, schema_registry_client, car_to_dict)

    producer = Producer(sasl_conf())

    print("Producing user records to topic {}. ^C to exit.".format(topic))
    #while True:
        # Serve on_delivery callbacks from previous calls to produce()
    producer.poll(0.0)
    try:
        for car in get_car_instance(file_path=FILE_PATH):

            print(car)
            producer.produce(topic=topic,
                            key=string_serializer(str(uuid4()), car_to_dict),
                            value=json_serializer(car, SerializationContext(topic, MessageField.VALUE)),
                            on_delivery=delivery_report)
            break
    except KeyboardInterrupt:
        pass
    except ValueError:
        print("Invalid input, discarding record...")
        pass

    print("\nFlushing records...")
    producer.flush()

main("test-topic-1")

Producing user records to topic test-topic-1. ^C to exit.
{'car_name': 'Maruti Alto', 'brand': 'Maruti', 'model': 'Alto', 'vehicle_age': 9, 'km_driven': 120000, 'seller_type': 'Individual', 'fuel_type': 'Petrol', 'transmission_type': 'Manual', 'mileage': 19.7, 'engine': 796, 'max_power': 46.3, 'seats': 5, 'selling_price': 120000}


SchemaRegistryError: Unauthorized (HTTP status code 401, SR code 401)

%3|1675579435.661|FAIL|rdkafka#producer-3| [thrd:sasl_ssl://pkc-6ojv2.us-west4.gcp.confluent.cloud:9092/bootstra]: sasl_ssl://pkc-6ojv2.us-west4.gcp.confluent.cloud:9092/bootstrap: SASL authentication error: Authentication failed (after 5301ms in state AUTH_REQ)
%3|1675579436.074|FAIL|rdkafka#producer-1| [thrd:sasl_ssl://pkc-lzvrd.us-west4.gcp.confluent.cloud:9092/bootstra]: sasl_ssl://pkc-lzvrd.us-west4.gcp.confluent.cloud:9092/bootstrap: SASL authentication error: Authentication failed (after 5214ms in state AUTH_REQ, 5 identical error(s) suppressed)
%3|1675579442.587|FAIL|rdkafka#producer-3| [thrd:sasl_ssl://pkc-6ojv2.us-west4.gcp.confluent.cloud:9092/bootstra]: sasl_ssl://pkc-6ojv2.us-west4.gcp.confluent.cloud:9092/bootstrap: SASL authentication error: Authentication failed (after 5222ms in state AUTH_REQ, 1 identical error(s) suppressed)
%3|1675579456.739|FAIL|rdkafka#producer-2| [thrd:sasl_ssl://pkc-6ojv2.us-west4.gcp.confluent.cloud:9092/bootstra]: sasl_ssl://pkc-6ojv2.us-west4.

%3|1675579751.077|FAIL|rdkafka#producer-1| [thrd:sasl_ssl://pkc-lzvrd.us-west4.gcp.confluent.cloud:9092/bootstra]: sasl_ssl://pkc-lzvrd.us-west4.gcp.confluent.cloud:9092/bootstrap: SASL authentication error: Authentication failed (after 5215ms in state AUTH_REQ, 5 identical error(s) suppressed)
%3|1675579757.594|FAIL|rdkafka#producer-3| [thrd:sasl_ssl://pkc-6ojv2.us-west4.gcp.confluent.cloud:9092/bootstra]: sasl_ssl://pkc-6ojv2.us-west4.gcp.confluent.cloud:9092/bootstrap: SASL authentication error: Authentication failed (after 5229ms in state AUTH_REQ, 5 identical error(s) suppressed)
%3|1675579771.730|FAIL|rdkafka#producer-2| [thrd:sasl_ssl://pkc-6ojv2.us-west4.gcp.confluent.cloud:9092/bootstra]: sasl_ssl://pkc-6ojv2.us-west4.gcp.confluent.cloud:9092/bootstrap: SASL authentication error: Authentication failed (after 5217ms in state AUTH_REQ, 5 identical error(s) suppressed)
%3|1675579786.072|FAIL|rdkafka#producer-1| [thrd:sasl_ssl://pkc-lzvrd.us-west4.gcp.confluent.cloud:9092/bootstra